In [1]:
import sys
sys.path.append('../src')

In [3]:
import gcsfs
fs = gcsfs.GCSFileSystem(
    project='research',
    bucket='amrit-research-samples',
    token='/Users/amrit/Desktop/StanfordAI/.keys/service-account.json'
)

## Load All Samples from GCS

In [5]:
from json import JSONDecodeError
import json
import tqdm

rows = []
for sample_url in tqdm.tqdm(fs.ls('amrit-research-samples/Mbpp')):
    contents = fs.cat(sample_url).decode('utf-8')
    lines = contents.strip('\n').split('\n')
    for line in lines:
        parsed_line = json.loads(line)
        
        for key in parsed_line['pass_at_original']:
            parsed_line[f'pass_at_original_{key}'] = parsed_line['pass_at_original'][key]
        
        for key in parsed_line['pass_at_mutated']:
            parsed_line[f'pass_at_mutated_{key}'] = parsed_line['pass_at_mutated'][key]
        
                
        for key in parsed_line['pass_at_ratio']:
            parsed_line[f'{key}_ratio'] = parsed_line['pass_at_ratio'][key]
    
        rows.append(parsed_line)

100%|██████████| 619/619 [01:51<00:00,  5.56it/s]


In [6]:
len(rows)

619

In [7]:
import pandas as pd

In [8]:
df = pd.DataFrame(rows)

In [9]:
df

,problem_id,mutation,mutation_id,stem_id,original_prefix,mutated_prefix,pass_at_original,pass_at_mutated,pass_at_ratio,average_levenshtein,examples,pass_at_original_1,pass_at_original_5,pass_at_original_10,pass_at_mutated_1,pass_at_mutated_5,pass_at_mutated_10,pass@1_ratio,pass@5_ratio,pass@10_ratio
0,Mbpp/100,AddParensTransformer,9,0,"def next_smallest_palindrome(num):\n """"""\n ...","def next_smallest_palindrome(num):\n """"""\n ...","{'1': 0.3768844221105532, '5': 0.9089294272097...","{'1': 0.4030612244897961, '5': 0.9268230952308...","{'pass@1': 1.0694529446977779, 'pass@5': 1.019...",56.356190,{'passed': {'original': ['def next_smallest_pa...,0.376884,0.908929,0.992349,0.403061,0.926823,0.995114,1.069453,1.019685,1.002786
1,Mbpp/100,AdditionInversionTransformer,30,0,"def next_smallest_palindrome(num):\n """"""\n ...","def next_smallest_palindrome(num):\n """"""\n ...","{'1': 1.0, '5': 1.0, '10': 1.0}","{'1': 0.29499999999999993, '5': 0.829513088880...","{'pass@1': 0.29499970500029493, 'pass@5': 0.82...",2.213559,{'passed': {'original': ['def next_smallest_pa...,1.000000,1.000000,1.000000,0.295000,0.829513,0.972498,0.295000,0.829512,0.972497
2,Mbpp/100,BlockCommentsTransformer,10,0,"def next_smallest_palindrome(num):\n """"""\n ...","def next_smallest_palindrome(num):\n """"""\n ...","{'1': 1.0, '5': 1.0, '10': 1.0}","{'1': 1.0, '5': 1.0, '10': 1.0}","{'pass@1': 0.9999990000010001, 'pass@5': 0.999...",54.704179,{'passed': {'original': ['def next_smallest_pa...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.999999,0.999999,0.999999
3,Mbpp/100,BlockCommentsTransformer,10,1,"def next_smallest_palindrome(num):\n """"""\n ...","def next_smallest_palindrome(num):\n """"""\n ...","{'1': 1.0, '5': 1.0, '10': 1.0}","{'1': 1.0, '5': 1.0, '10': 1.0}","{'pass@1': 0.9999990000010001, 'pass@5': 0.999...",62.000000,{'passed': {'original': ['def next_smallest_pa...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.999999,0.999999,0.999999
4,Mbpp/100,BlockCommentsTransformer,10,2,"def next_smallest_palindrome(num):\n """"""\n ...","def next_smallest_palindrome(num):\n """"""\n ...","{'1': 0.9099999999999999, '5': 0.9999966209848...","{'1': 0.4450000000000002, '5': 0.9494548912593...","{'pass@1': 0.48901045163686657, 'pass@5': 0.94...",79.259925,{'passed': {'original': ['def next_smallest_pa...,0.910000,0.999997,1.000000,0.445000,0.949455,0.997704,0.489010,0.949457,0.997703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,Mbpp/771,SingleElementInitializerTransformer,4,0,"def check_expression(exp):\n """"""\n Write...","def check_expression(exp):\n """"""\n Write...","{'1': 0.4199999999999998, '5': 0.9367462049297...","{'1': 0.4900000000000001, '5': 0.9671517485906...","{'pass@1': 1.1666638888955034, 'pass@5': 1.032...",125.683734,{'passed': {'original': ['def check_expression...,0.420000,0.936746,0.996367,0.490000,0.967152,0.999051,1.166664,1.032458,1.002693
615,Mbpp/771,UnusedVariableTransformer,18,0,"def check_expression(exp):\n """"""\n Write...","def check_expression(exp):\n """"""\n Write...","{'1': 1.0, '5': 1.0, '10': 1.0}","{'1': 1.0, '5': 1.0, '10': 1.0}","{'pass@1': 0.9999990000010001, 'pass@5': 0.999...",27.616169,{'passed': {'original': ['def check_expression...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.999999,0.999999,0.999999
616,Mbpp/771,UnusedVariableTransformer,18,1,"def check_expression(exp):\n """"""\n Write...","def check_expression(exp):\n """"""\n Write...","{'1': 1.0, '5': 1.0, '10': 1.0}","{'1': 1.0, '5': 1.0, '10': 1.0}","{'pass@1': 0.9999990000010001, 'pass@5': 0.999...",27.534129,{'passed': {'original': ['def check_expression...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.999999,0.999999,0.999999
617,Mbpp/771,UnusedVariableTransformer,18,2,"def check_expression(exp):\n """"""\n Write...","def check_expression(exp):\n """"""\n Write...","{'1': 0.495, '5': 0.9687619569930872, '10': 0....","{'1': 0.8700000000000001, '5': 0.9999740579342.

## Average Cross Levenshtein Distance by Mutation 

In [10]:
df.groupby('mutation')['average_levenshtein'].mean()

mutation
AddParensTransformer                    12.248687
AdditionInversionTransformer             8.206903
ArrayToDictTransformer                 191.833333
BlockCommentsTransformer                80.972842
DictInitializerUnpackTransformer        38.671194
EmptyArrayToStringTransformer          183.340976
EnumerateForTransformer                 36.132877
ExpandAugmentedAssignTransformer         9.183971
ExpandBooleansTransformer               41.187178
FirstInversionTransformer               42.328066
ForToWhileTransformer                   65.874742
IdentifierObfuscateTransformer          64.230863
IdentifierRenameTransformer             41.721350
IdentityAssignmentTransformer           38.388711
IfToWhileLoopTransformer                20.376482
InlineCommentsTransformer               36.689623
IntegerBinTransformer                   18.474669
IntegerHexTransformer                   21.960493
IntegerOctTransformer                   18.898987
IntegerReplacementTransformer           1

In [20]:
df.groupby('mutation')['pass@1_ratio'].median().sort_values(ascending=False)


mutation
ListInitializerUnpackTransformer       1.813411
EmptyArrayToStringTransformer          1.374997
SingleElementInitializerTransformer    1.166664
AddParensTransformer                   0.999999
IdentityAssignmentTransformer          0.999999
SecondInversionTransformer             0.999999
ModuloInversionTransformer             0.999999
LenToGeneratorTransformer              0.999999
IntegerReplacementTransformer          0.999999
IntegerOctTransformer                  0.999999
IntegerHexTransformer                  0.999999
InlineCommentsTransformer              0.999999
IntegerBinTransformer                  0.999999
IdentifierRenameTransformer            0.999999
ExpandBooleansTransformer              0.999999
BlockCommentsTransformer               0.999999
EnumerateForTransformer                0.999999
ExpandAugmentedAssignTransformer       0.999999
DictInitializerUnpackTransformer       0.999999
ForToWhileTransformer                  0.999999
PrintInjectionTransformer      

In [ ]:
a